In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd
additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

DATABASE_NAME = "cbr_database_15_70"
OUTPUT_FILE = "cbr_15_70_gpt2_input"

IX_FILE1 = "cbr_augmentation_1_gpt2_results_p_ix.csv"
IX_FILE2 = "cbr_augmentation_2_gpt2_results_p_ix.csv"
dfix1 = pd.read_csv(f"{IX_FILE1}")
dfix2 = pd.read_csv(f"{IX_FILE2}")
cbr_database = cbr_database[:int(len(cbr_database)/2)]
cbr_id = cbr_database.set_index("id")

In [2]:
cbr_db = pd.concat([cbr_database, additional_cbr_1, additional_cbr_2])
cbr_db_ix = pd.concat([cbr_database, dfix1, dfix2])
cbr_db_ix = cbr_db_ix.set_index("id")

large_cbr_reps = str_rep_vdb(cbr_db)
len(large_cbr_reps)

15939

In [3]:
cbr_db_ix

,name,steps,ingredients
id,,,
74113,creamy swiss chicken breasts,Arrange chicken in a 13x9x2 inch baking dish t...,"chicken breasts, swiss cheese, low-fat cream o..."
257796,slow cooker garlic chicken with rosemary,"Place rosemary springs, 1 lemon half, celery, ...","roasting chickens, lemons, rosemary sprigs, pa..."
251645,curried chicken rice salad,"Combine cool rice, chicken, pineapple, celery ...","raw rice, cooked chicken, canned pineapple, ce..."
389514,omelet with corn and smoked mozzarella,Cut corn from cob or use frozen and place in a...,"fresh corn, water, eggs, salt & freshly ground..."
349283,chilled peach soup with riesling,"In a large saucepan, combine the riesling, clo...","riesling wine, whole cloves, cinnamon stick, b..."
...,...,...,...
122286,south carolina strawberry shortcake,Preheat oven to 350f line a 12-cup muffin tin ...,"all-purpose flour, baking powder, salt, butter..."
53152,souper easy macaroni and cheese,"In dutch oven or electric skillet, heat oil. A...","nacho cheese soup, cheddar cheese, milk, prepa..."
458446,house of blues ginger lemongrass soda water,"In a medium saucepan, combine 2 cups water, su...","water, sugar, lemongrass, fresh ginger, lemon,..."


In [4]:
# large_cbr = pd.concat([cbr_database, cbr_augmentation, additional_cbr_1, additional_cbr_2])
# large_cbr_id = large_cbr.set_index("id")
# large_cbr_reps = str_rep_vdb(large_cbr_id)
# len(large_cbr_reps)

In [5]:
meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_db["id"].values)], {"dataset":"cbr_15_70"})
vdb = CBRDatabase(large_cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

# vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)
print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=1))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
15939
[Document(page_content='name: never fail banana bread ; ingredients: flour, margarine, bananas, nuts, salt, sugar, eggs, baking soda ; preparation:', metadata={'dataset': 'cbr_15_70', 'recipe_id': 123543})]


In [6]:
# generowanie inputów - 850 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        tem = vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
        # print(tem)#dev
        # print(compose(cbr_db_ix.loc[[recipe_ixs[0]]], v_recipe))#dev
        # raise

        gen_input = compose(cbr_db_ix.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(4, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(3, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(2, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+3:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_db_ix.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 733
1 - 784
1 - 714
1 - 813
1 - 753
1 - 584
1 - 767
1 - 798
1 - 644
1 - 664
1 - 694
1 - 712
1 - 506
1 - 775
1 - 812
1 - 612
1 - 756
1 - 691
1 - 829
1 - 802
1 - 750
1 - 801
1 - 679
1 - 725
1 - 723
1 - 726
1 - 695
1 - 749
1 - 705
1 - 687
1 - 728
1 - 719
1 - 787
1 - 741
1 - 747
1 - 733
2 - 444
1 - 722
1 - 746
1 - 741
1 - 727
1 - 726
1 - 624
1 - 733
1 - 766
1 - 850
1 - 782
1 - 828
1 - 753
1 - 720
1 - 792
1 - 711
1 - 655
1 - 799
1 - 818
1 - 701
1 - 738
1 - 814
1 - 732
1 - 777
1 - 806
1 - 676
1 - 713
1 - 712
1 - 745
1 - 785
1 - 745
1 - 657
1 - 738
2 - 716
2 - 756
1 - 712
1 - 788
2 - 488
1 - 705
1 - 729
1 - 750
1 - 760
1 - 832
1 - 781
1 - 696
1 - 806
1 - 806
1 - 646
2 - 727
1 - 685
1 - 812
1 - 758
1 - 796
1 - 678
1 - 655
1 - 677
1 - 724
1 - 730
1 - 753
1 - 711
2 - 698
1 - 665
1 - 836
1 - 796
1 - 712
1 - 847
1 - 742
1 - 674
1 - 833
1 - 708
1 - 769
1 - 790
1 - 789
1 - 748
1 - 697
1 - 704
1 - 636
1 - 718
1 - 787
1 - 605
1 - 780
1 - 718
1 - 762
1 - 770
1 - 527
1 - 736
1 - 804
1 - 850
1 - 786


Token indices sequence length is longer than the specified maximum sequence length for this model (2496 > 2048). Running this sequence through the model will result in indexing errors


1 - 807
1 - 823
1 - 764
1 - 825
1 - 778
2 - 282
1 - 774
1 - 770
1 - 748
1 - 782
1 - 758
1 - 771
1 - 701
1 - 717
1 - 678
1 - 769
1 - 824
1 - 761
1 - 742
1 - 651
1 - 568
1 - 805
1 - 748
1 - 754
1 - 747
1 - 634
1 - 772
1 - 652
1 - 677
1 - 736
1 - 764
1 - 763
1 - 666
1 - 743
1 - 834
1 - 714
1 - 768
1 - 650
1 - 803
1 - 669
1 - 696
1 - 782
1 - 802
1 - 589
1 - 591
1 - 781
1 - 789
1 - 731
1 - 721
1 - 641
1 - 822
1 - 708
1 - 695
1 - 709
1 - 775
1 - 640
1 - 646
1 - 781
1 - 402
1 - 722
1 - 756
1 - 793
1 - 665
1 - 729
1 - 730
1 - 693
1 - 787
1 - 541
1 - 793
1 - 757
1 - 751
1 - 667
1 - 791
1 - 672
1 - 647
1 - 715
1 - 750
1 - 668
1 - 722
1 - 797
1 - 785
1 - 730
1 - 737
1 - 718
1 - 778
1 - 728
1 - 610
1 - 771
1 - 773
1 - 654
1 - 823
1 - 717
1 - 798
1 - 836
1 - 700
1 - 810
1 - 751
2 - 832
1 - 795
1 - 717
1 - 810
1 - 718
1 - 847
1 - 733
1 - 849
1 - 652
1 - 709
1 - 681
1 - 671
1 - 734
1 - 724
1 - 725
1 - 730
1 - 725
1 - 737
1 - 708
1 - 713
1 - 728
1 - 728
1 - 753
1 - 755
1 - 730
1 - 701
1 - 776
1 - 676


In [7]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}.csv", index=False)

In [8]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=7)

[Document(page_content='name: easy pressure cooker roast with gravy ; ingredients: chuck roast, onion, baby carrots, brown gravy mix, beef stock, garlic powder, black pepper ; preparation:', metadata={'dataset': 'cbr_15_70', 'recipe_id': 279036}),
 Document(page_content='name: roasted garlic make ahead mashed potatoes ; ingredients: potatoes, chicken broth, cream cheese, roasted garlic clove, salt and pepper, parmesan cheese ; preparation:', metadata={'dataset': 'cbr_15_70', 'recipe_id': 232332}),
 Document(page_content='name: a mountain of make ahead mashed potatoes ; ingredients: potatoes, sour cream, cream cheese, butter, dried onion flakes, salt, paprika ; preparation:', metadata={'dataset': 'cbr_15_70', 'recipe_id': 88831}),
 Document(page_content='name: slow cooker pot roast ; ingredients: cornstarch, cold water, carrots, onions, coarse salt, ground black pepper, chuck roast, worcestershire sauce ; preparation:', metadata={'dataset': 'cbr_15_70', 'recipe_id': 327293}),
 Document(

In [9]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1089, 7: 996, 5: 37, 4: 3})